In [20]:
import os
import numpy as np
import pandas as pd
import json
import random
import warnings
from itertools import chain
from tqdm import tqdm
import torch
import torch.nn.functional as F
import torch.nn.parallel
from torch import Tensor, nn
from torch.overrides import has_torch_function, handle_torch_function
from typing import Optional, List, Tuple
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sksurv.metrics import concordance_index_censored

In [21]:
class CoAttnData(Dataset):
    def __init__(self, task: List, codebook_dict: dict, num_bins=4, fold=0, mode="train"):
        super(CoAttnData, self).__init__()

        self.fold = fold
        self.bins = num_bins
        self.name_dict = codebook_dict

        all_patients = []
        all_labels = pd.DataFrame()
        all_config_dicts = {}

        for task_name in sorted(task):
            # dataset split, combining patients in all tasks
            json_path = f"/dssg/home/acct-clsyzs/clsyzs/xiayujia/CoPaRa/coattention_prognosis/" \
                        f"splits/json_{task_name.lower()}/"
            with open(os.path.join(json_path, f"fold_{self.fold}.json"), "r") as f:
                task_patients = json.load(f)[mode]
            all_patients.append(task_patients)

            # get discrete survival label
            task_label_path = "/dssg/home/acct-clsyzs/clsyzs/xiayujia/CoPaRa/coattention_prognosis/preparation/" \
                              f"clinical_info_clean_{task_name.lower()}.csv"
            task_labels = pd.read_csv(task_label_path, index_col="patient")
            all_labels = pd.concat([all_labels, task_labels], axis=0)

            # get radiological path and pathological path
            # define pathomics path generated by pretraining graph
            pathomics_merge_base = f"/dssg/home/acct-clsyzs/clsyzs/xiayujia/CoPaRa_data/TCGA-{task_name}/patch_features_merged_resnet/"

            # define radiomics path generated by PyRadiomics
            radiomics_base = f"/dssg/home/acct-clsyzs/clsyzs/xiayujia/CoPaRa_data/TRAD-{task_name}/summary_radiomics.csv"
            radiomics_data = pd.read_csv(radiomics_base).set_index("imageFile").loc[task_patients, :]
            all_columns = list(radiomics_data.columns)
            diagnostic_cols = sorted(list(filter(lambda x: "diagnostics" in x, all_columns)))
            shape_cols = sorted(list(filter(lambda x: "original_shape" in x, all_columns)))
            order_cols = sorted(list(filter(lambda x: "original_firstorder" in x, all_columns)))
            texture_cols = sorted(list(filter(lambda x: ("original_glcm" in x) or ("original_gldm" in x)
                                                        or ("original_glrlm" in x) or ("original_glszm" in x)
                                                        or ("original_ngtdm" in x), all_columns)))
            log_cols = sorted(list(filter(lambda x: "log-sigma" in x, all_columns)))
            wavelet_cols = sorted(list(filter(lambda x: "wavelet" in x, all_columns)))
            assert len(all_columns) == len(diagnostic_cols) + len(shape_cols) + len(order_cols) \
                   + len(texture_cols) + len(log_cols) + len(wavelet_cols)

            # perform ss for radiomics data
            radiomics_data = radiomics_data.drop(diagnostic_cols, axis=1)
            ss = StandardScaler()
            radio_data = pd.DataFrame(ss.fit_transform(radiomics_data),
                                      index=radiomics_data.index,
                                      columns=radiomics_data.columns)
            shape = radio_data[shape_cols]
            order = radio_data[order_cols]
            texture = radio_data[texture_cols]
            log_sigma = radio_data[log_cols]
            wavelet = radio_data[wavelet_cols]

            # define radiomics path generated by RadFM
            radio_cnn_path = f"/dssg/home/acct-clsyzs/clsyzs/xiayujia/CoPaRa_data/TRAD-{task_name}/ct_nii_features/"

            all_config_dicts[task_name] = {"pathomics": pathomics_merge_base,
                                           "shape": shape, "order": order, "texture": texture, "log_sigma": log_sigma,
                                           "wavelet": wavelet, "radfm": radio_cnn_path}
        self.config = all_config_dicts
        self.patients = list(chain(*all_patients))
        self.labels = all_labels

    def __len__(self):
        return len(self.patients)

    def __getitem__(self, item):
        patient_name = self.patients[item]
        task_id = self.name_dict[patient_name]
        config_dict = self.config[task_id]

        labels = self.labels.loc[patient_name]
        surv_discrete = labels["time_bins"]
        surv_time = labels["time"]
        censor = 1 - labels["event"]

        # pathological data
        patho = torch.load(os.path.join(config_dict["pathomics"], f"{patient_name}.pt")).to(torch.float32)

        # radiological data
        radio_shape = torch.from_numpy(config_dict["shape"].loc[patient_name, :].values).to(torch.float32)
        radio_order = torch.from_numpy(config_dict["order"].loc[patient_name, :].values).to(torch.float32)
        radio_texture = torch.from_numpy(config_dict["texture"].loc[patient_name, :].values).to(torch.float32)
        radio_log_sigma = torch.from_numpy(config_dict["log_sigma"].loc[patient_name, :].values).to(torch.float32)
        radio_wavelet = torch.from_numpy(config_dict["wavelet"].loc[patient_name, :].values).to(torch.float32)

        # RadFM data
        radio_cnn = torch.load(os.path.join(config_dict["radfm"], f"{patient_name}.pt")).squeeze(0).to(torch.float32)

        data = [patho, radio_shape, radio_order, radio_texture, radio_log_sigma, radio_wavelet, radio_cnn,
                surv_discrete, surv_time, censor, patient_name]

        return data

In [22]:
def SNN_Block(dim1, dim2, dropout=0.25):
    r"""
    Multilayer Reception Block w/ Self-Normalization (Linear + ELU + Alpha Dropout)

    args:
        dim1 (int): Dimension of input features
        dim2 (int): Dimension of output features
        dropout (float): Dropout rate
    """
    import torch.nn as nn

    return nn.Sequential(
        nn.Linear(dim1, dim2),
        nn.ELU(),
        nn.AlphaDropout(p=dropout, inplace=False))


class Attn_Net_Gated(nn.Module):
    def __init__(self, L=1024, D=256, dropout=False, n_classes=1):
        r"""
        Attention Network with Sigmoid Gating (3 fc layers)

        args:
            L (int): input feature dimension
            D (int): hidden layer dimension
            dropout (bool): whether to apply dropout (p = 0.25)
            n_classes (int): number of classes
        """
        super(Attn_Net_Gated, self).__init__()
        self.attention_a = [
            nn.Linear(L, D),
            nn.Tanh()]

        self.attention_b = [nn.Linear(L, D), nn.Sigmoid()]
        if dropout:
            self.attention_a.append(nn.Dropout(0.25))
            self.attention_b.append(nn.Dropout(0.25))

        self.attention_a = nn.Sequential(*self.attention_a)
        self.attention_b = nn.Sequential(*self.attention_b)
        self.attention_c = nn.Linear(D, n_classes)

    def forward(self, x):
        a = self.attention_a(x)
        b = self.attention_b(x)
        A = a.mul(b)
        A = self.attention_c(A)  # N x n_classes
        return A, x

In [23]:
class PathoBlock(nn.Module):
    def __init__(self, size_embed, dropout):
        super(PathoBlock, self).__init__()
        path_encoder_layer = nn.TransformerEncoderLayer(d_model=size_embed, nhead=8, dim_feedforward=512,
                                                        dropout=dropout, activation='relu')
        self.path_transformer = nn.TransformerEncoder(path_encoder_layer, num_layers=2)
        self.path_attention_head = Attn_Net_Gated(L=size_embed, D=size_embed,
                                                  dropout=dropout, n_classes=1)
        self.path_rho = nn.Sequential(*[nn.Linear(size_embed, size_embed),
                                        nn.ReLU(), nn.Dropout(dropout)])

    def forward(self, x):
        h_path_trans = self.path_transformer(x)
        A_path, h_path = self.path_attention_head(h_path_trans.squeeze(1))
        A_path = torch.transpose(A_path, 1, 0)
        h_path = torch.mm(F.softmax(A_path, dim=1), h_path)
        h_path = self.path_rho(h_path).squeeze()
        return h_path, A_path


class RadioBlock(nn.Module):
    def __init__(self, size_embed, dropout):
        super(RadioBlock, self).__init__()
        omic_encoder_layer = nn.TransformerEncoderLayer(d_model=size_embed, nhead=8, dim_feedforward=512,
                                                        dropout=dropout, activation='relu')
        self.omic_transformer = nn.TransformerEncoder(omic_encoder_layer, num_layers=2)
        self.omic_attention_head = Attn_Net_Gated(L=size_embed, D=size_embed,
                                                  dropout=dropout, n_classes=1)
        self.omic_rho = nn.Sequential(*[nn.Linear(size_embed, size_embed),
                                        nn.ReLU(), nn.Dropout(dropout)])

    def forward(self, x):
        h_omic_trans = self.omic_transformer(x)
        A_omic, h_omic = self.omic_attention_head(h_omic_trans.squeeze(1))
        A_omic = torch.transpose(A_omic, 1, 0)
        h_omic = torch.mm(F.softmax(A_omic, dim=1), h_omic)
        h_omic = self.omic_rho(h_omic).squeeze()
        return h_omic, A_omic


class FusionBlock(nn.Module):
    def __init__(self, size_embed, fusion, n_classes):
        super(FusionBlock, self).__init__()

        self.fusion = fusion
        self.mm = nn.Sequential(*[nn.Linear(size_embed * 2, size_embed), nn.ReLU(),
                                  nn.Linear(size_embed, size_embed),
                                  nn.ReLU()])
        self.classifier = nn.Linear(size_embed, n_classes)

    def forward(self, h_path, h_omic):
        h = self.mm(torch.cat([h_path, h_omic], dim=0))
        logits = self.classifier(h).unsqueeze(0)
        Y_hat = torch.topk(logits, 1, dim=1)[1]
        hazards = torch.sigmoid(logits)
        S = torch.cumprod(1 - hazards, dim=1)

        return logits, Y_hat, hazards, S

In [24]:
class PaRaMILUnionInfer(nn.Module):
    def __init__(self, fusion='concat', tasks: List = ["SHRJ", "LUNG"],
                 n_classes=4, dropout=0.25, codebook_dict: dict = {"a": "b"}):
        super(PaRaMILUnionInfer, self).__init__()
        self.omic_sizes = [14, 18, 75, 279, 744, 768]  # radiomics sizes
        self.n_classes = n_classes
        self.tasks = tasks
        self.permit_name_dict = codebook_dict

        # pathomics bag formulation
        self.wsi_net = nn.Sequential(*[nn.Linear(512, 192),
                                       nn.ReLU(),
                                       nn.Dropout(0.25)])
        # radiomics bag formulation
        sig_networks = []
        for input_dim in self.omic_sizes:
            sig_networks.append(nn.Sequential(*[SNN_Block(dim1=input_dim, dim2=256),
                                                SNN_Block(dim1=256, dim2=256, dropout=0.25)]))
        self.sig_networks = nn.ModuleList(sig_networks)
        # coattention module
        size_embed = 256
        self.coattn = MultiheadAttention(embed_dim=size_embed, num_heads=1)

        # Path Transformer + Attention Head
        self.patho_block = nn.ModuleList(PathoBlock(size_embed, dropout) for i in range(len(self.tasks)))

        # Omic Transformer + Attention Head
        self.radio_block = nn.ModuleList(RadioBlock(size_embed, dropout) for i in range(len(self.tasks)))

        # Fusion Layer & Classifier
        self.fusion_block = nn.ModuleList(FusionBlock(size_embed, fusion, n_classes) for i in range(len(self.tasks)))

    def forward(self, **kwargs):
        x_path = kwargs['x_path']
        x_omic = [kwargs['x_omic%d' % i] for i in range(1, 7)]
        patient_name = kwargs["patient_name"]

        h_path_bag = self.wsi_net(x_path[:, :, :512])
        h_path_bag = torch.concat([h_path_bag, x_path[:, :, 512:]], dim=2)

        h_omic = [self.sig_networks[idx].forward(sig_feat) for idx, sig_feat in enumerate(x_omic)]
        h_omic_bag = torch.stack(h_omic)

        # Coattn
        h_path_coattn, A_coattn = self.coattn(h_omic_bag, h_path_bag, h_path_bag)

        permit_name_dict = self.permit_name_dict
        task_id = permit_name_dict[patient_name[0]]
        task_locs = [i for i, item in enumerate(self.tasks) if item == task_id]
        assert len(task_locs) == 1
        task_loc = task_locs[0]

        h_path, A_path = self.patho_block[task_loc](h_path_coattn)  # Pathology
        h_omic, A_omic = self.radio_block[task_loc](h_omic_bag)  # Radiology

        logits, Y_hat, hazards, S = self.fusion_block[task_loc](h_path, h_omic)  # Classifier

        attention_scores = {'coattn': A_coattn, 'path': A_path, 'omic': A_omic}

        return hazards, S, Y_hat, attention_scores, h_path_coattn, h_omic_bag


###
# ========== Modifying PyTorch Functionalities ======================
###
from torch.nn.functional import *


def multi_head_attention_forward(
        query: Tensor,
        key: Tensor,
        value: Tensor,
        embed_dim_to_check: int,
        num_heads: int,
        in_proj_weight: Tensor,
        in_proj_bias: Tensor,
        bias_k: Optional[Tensor],
        bias_v: Optional[Tensor],
        add_zero_attn: bool,
        dropout_p: float,
        out_proj_weight: Tensor,
        out_proj_bias: Tensor,
        training: bool = True,
        key_padding_mask: Optional[Tensor] = None,
        need_weights: bool = True,
        need_raw: bool = True,
        attn_mask: Optional[Tensor] = None,
        use_separate_proj_weight: bool = False,
        q_proj_weight: Optional[Tensor] = None,
        k_proj_weight: Optional[Tensor] = None,
        v_proj_weight: Optional[Tensor] = None,
        static_k: Optional[Tensor] = None,
        static_v: Optional[Tensor] = None,
):
    r"""
    Args:
        query, key, value: map a query and a set of key-value pairs to an output.
            See "Attention Is All You Need" for more details.
        embed_dim_to_check: total dimension of the model.
        num_heads: parallel attention heads.
        in_proj_weight, in_proj_bias: input projection weight and bias.
        bias_k, bias_v: bias of the key and value sequences to be added at dim=0.
        add_zero_attn: add a new batch of zeros to the key and
                       value sequences at dim=1.
        dropout_p: probability of an element to be zeroed.
        out_proj_weight, out_proj_bias: the output projection weight and bias.
        training: apply dropout if is ``True``.
        key_padding_mask: if provided, specified padding elements in the key will
            be ignored by the attention. This is an binary mask. When the value is True,
            the corresponding value on the attention layer will be filled with -inf.
        need_weights: output attn_output_weights.
        attn_mask: 2D or 3D mask that prevents attention to certain positions. A 2D mask will be broadcasted for all
            the batches while a 3D mask allows to specify a different mask for the entries of each batch.
        use_separate_proj_weight: the function accept the proj. weights for query, key,
            and value in different forms. If false, in_proj_weight will be used, which is
            a combination of q_proj_weight, k_proj_weight, v_proj_weight.
        q_proj_weight, k_proj_weight, v_proj_weight, in_proj_bias: input projection weight and bias.
        static_k, static_v: static key and value used for attention operators.
    Shape:
        Inputs:
        - query: :math:`(L, N, E)` where L is the target sequence length, N is the batch size, E is
          the embedding dimension.
        - key: :math:`(S, N, E)`, where S is the source sequence length, N is the batch size, E is
          the embedding dimension.
        - value: :math:`(S, N, E)` where S is the source sequence length, N is the batch size, E is
          the embedding dimension.
        - key_padding_mask: :math:`(N, S)` where N is the batch size, S is the source sequence length.
          If a ByteTensor is provided, the non-zero positions will be ignored while the zero positions
          will be unchanged. If a BoolTensor is provided, the positions with the
          value of ``True`` will be ignored while the position with the value of ``False`` will be unchanged.
        - attn_mask: 2D mask :math:`(L, S)` where L is the target sequence length, S is the source sequence length.
          3D mask :math:`(N*num_heads, L, S)` where N is the batch size, L is the target sequence length,
          S is the source sequence length. attn_mask ensures that position i is allowed to attend the unmasked
          positions. If a ByteTensor is provided, the non-zero positions are not allowed to attend
          while the zero positions will be unchanged. If a BoolTensor is provided, positions with ``True``
          are not allowed to attend while ``False`` values will be unchanged. If a FloatTensor
          is provided, it will be added to the attention weight.
        - static_k: :math:`(N*num_heads, S, E/num_heads)`, where S is the source sequence length,
          N is the batch size, E is the embedding dimension. E/num_heads is the head dimension.
        - static_v: :math:`(N*num_heads, S, E/num_heads)`, where S is the source sequence length,
          N is the batch size, E is the embedding dimension. E/num_heads is the head dimension.
        Outputs:
        - attn_output: :math:`(L, N, E)` where L is the target sequence length, N is the batch size,
          E is the embedding dimension.
        - attn_output_weights: :math:`(N, L, S)` where N is the batch size,
          L is the target sequence length, S is the source sequence length.
    """
    tens_ops = (query, key, value, in_proj_weight, in_proj_bias, bias_k, bias_v, out_proj_weight, out_proj_bias)
    if has_torch_function(tens_ops):
        return handle_torch_function(
            multi_head_attention_forward,
            tens_ops,
            query,
            key,
            value,
            embed_dim_to_check,
            num_heads,
            in_proj_weight,
            in_proj_bias,
            bias_k,
            bias_v,
            add_zero_attn,
            dropout_p,
            out_proj_weight,
            out_proj_bias,
            training=training,
            key_padding_mask=key_padding_mask,
            need_weights=need_weights,
            need_raw=need_raw,
            attn_mask=attn_mask,
            use_separate_proj_weight=use_separate_proj_weight,
            q_proj_weight=q_proj_weight,
            k_proj_weight=k_proj_weight,
            v_proj_weight=v_proj_weight,
            static_k=static_k,
            static_v=static_v,
        )
    tgt_len, bsz, embed_dim = query.size()
    assert embed_dim == embed_dim_to_check
    # allow MHA to have different sizes for the feature dimension
    assert key.size(0) == value.size(0) and key.size(1) == value.size(1)

    head_dim = embed_dim // num_heads
    assert head_dim * num_heads == embed_dim, "embed_dim must be divisible by num_heads"
    scaling = float(head_dim) ** -0.5

    if not use_separate_proj_weight:
        if (query is key or torch.equal(query, key)) and (key is value or torch.equal(key, value)):
            # self-attention
            q, k, v = linear(query, in_proj_weight, in_proj_bias).chunk(3, dim=-1)

        elif key is value or torch.equal(key, value):
            # encoder-decoder attention
            # This is inline in_proj function with in_proj_weight and in_proj_bias
            _b = in_proj_bias
            _start = 0
            _end = embed_dim
            _w = in_proj_weight[_start:_end, :]
            if _b is not None:
                _b = _b[_start:_end]
            q = linear(query, _w, _b)

            if key is None:
                assert value is None
                k = None
                v = None
            else:

                # This is inline in_proj function with in_proj_weight and in_proj_bias
                _b = in_proj_bias
                _start = embed_dim
                _end = None
                _w = in_proj_weight[_start:, :]
                if _b is not None:
                    _b = _b[_start:]
                k, v = linear(key, _w, _b).chunk(2, dim=-1)

        else:
            # This is inline in_proj function with in_proj_weight and in_proj_bias
            _b = in_proj_bias
            _start = 0
            _end = embed_dim
            _w = in_proj_weight[_start:_end, :]
            if _b is not None:
                _b = _b[_start:_end]
            q = linear(query, _w, _b)

            # This is inline in_proj function with in_proj_weight and in_proj_bias
            _b = in_proj_bias
            _start = embed_dim
            _end = embed_dim * 2
            _w = in_proj_weight[_start:_end, :]
            if _b is not None:
                _b = _b[_start:_end]
            k = linear(key, _w, _b)

            # This is inline in_proj function with in_proj_weight and in_proj_bias
            _b = in_proj_bias
            _start = embed_dim * 2
            _end = None
            _w = in_proj_weight[_start:, :]
            if _b is not None:
                _b = _b[_start:]
            v = linear(value, _w, _b)
    else:
        q_proj_weight_non_opt = torch.jit._unwrap_optional(q_proj_weight)
        len1, len2 = q_proj_weight_non_opt.size()
        assert len1 == embed_dim and len2 == query.size(-1)

        k_proj_weight_non_opt = torch.jit._unwrap_optional(k_proj_weight)
        len1, len2 = k_proj_weight_non_opt.size()
        assert len1 == embed_dim and len2 == key.size(-1)

        v_proj_weight_non_opt = torch.jit._unwrap_optional(v_proj_weight)
        len1, len2 = v_proj_weight_non_opt.size()
        assert len1 == embed_dim and len2 == value.size(-1)

        if in_proj_bias is not None:
            q = linear(query, q_proj_weight_non_opt, in_proj_bias[0:embed_dim])
            k = linear(key, k_proj_weight_non_opt, in_proj_bias[embed_dim: (embed_dim * 2)])
            v = linear(value, v_proj_weight_non_opt, in_proj_bias[(embed_dim * 2):])
        else:
            q = linear(query, q_proj_weight_non_opt, in_proj_bias)
            k = linear(key, k_proj_weight_non_opt, in_proj_bias)
            v = linear(value, v_proj_weight_non_opt, in_proj_bias)
    q = q * scaling

    if attn_mask is not None:
        assert (
                attn_mask.dtype == torch.float32
                or attn_mask.dtype == torch.float64
                or attn_mask.dtype == torch.float16
                or attn_mask.dtype == torch.uint8
                or attn_mask.dtype == torch.bool
        ), "Only float, byte, and bool types are supported for attn_mask, not {}".format(attn_mask.dtype)
        if attn_mask.dtype == torch.uint8:
            warnings.warn("Byte tensor for attn_mask in nn.MultiheadAttention is deprecated. Use bool tensor instead.")
            attn_mask = attn_mask.to(torch.bool)

        if attn_mask.dim() == 2:
            attn_mask = attn_mask.unsqueeze(0)
            if list(attn_mask.size()) != [1, query.size(0), key.size(0)]:
                raise RuntimeError("The size of the 2D attn_mask is not correct.")
        elif attn_mask.dim() == 3:
            if list(attn_mask.size()) != [bsz * num_heads, query.size(0), key.size(0)]:
                raise RuntimeError("The size of the 3D attn_mask is not correct.")
        else:
            raise RuntimeError("attn_mask's dimension {} is not supported".format(attn_mask.dim()))
        # attn_mask's dim is 3 now.

    # convert ByteTensor key_padding_mask to bool
    if key_padding_mask is not None and key_padding_mask.dtype == torch.uint8:
        warnings.warn(
            "Byte tensor for key_padding_mask in nn.MultiheadAttention is deprecated. Use bool tensor instead."
        )
        key_padding_mask = key_padding_mask.to(torch.bool)

    if bias_k is not None and bias_v is not None:
        if static_k is None and static_v is None:
            k = torch.cat([k, bias_k.repeat(1, bsz, 1)])
            v = torch.cat([v, bias_v.repeat(1, bsz, 1)])
            if attn_mask is not None:
                attn_mask = pad(attn_mask, (0, 1))
            if key_padding_mask is not None:
                key_padding_mask = pad(key_padding_mask, (0, 1))
        else:
            assert static_k is None, "bias cannot be added to static key."
            assert static_v is None, "bias cannot be added to static value."
    else:
        assert bias_k is None
        assert bias_v is None

    q = q.contiguous().view(tgt_len, bsz * num_heads, head_dim).transpose(0, 1)
    if k is not None:
        k = k.contiguous().view(-1, bsz * num_heads, head_dim).transpose(0, 1)
    if v is not None:
        v = v.contiguous().view(-1, bsz * num_heads, head_dim).transpose(0, 1)

    if static_k is not None:
        assert static_k.size(0) == bsz * num_heads
        assert static_k.size(2) == head_dim
        k = static_k

    if static_v is not None:
        assert static_v.size(0) == bsz * num_heads
        assert static_v.size(2) == head_dim
        v = static_v

    src_len = k.size(1)

    if key_padding_mask is not None:
        assert key_padding_mask.size(0) == bsz
        assert key_padding_mask.size(1) == src_len

    if add_zero_attn:
        src_len += 1
        k = torch.cat([k, torch.zeros((k.size(0), 1) + k.size()[2:], dtype=k.dtype, device=k.device)], dim=1)
        v = torch.cat([v, torch.zeros((v.size(0), 1) + v.size()[2:], dtype=v.dtype, device=v.device)], dim=1)
        if attn_mask is not None:
            attn_mask = pad(attn_mask, (0, 1))
        if key_padding_mask is not None:
            key_padding_mask = pad(key_padding_mask, (0, 1))

    attn_output_weights = torch.bmm(q, k.transpose(1, 2))
    assert list(attn_output_weights.size()) == [bsz * num_heads, tgt_len, src_len]

    if attn_mask is not None:
        if attn_mask.dtype == torch.bool:
            attn_output_weights.masked_fill_(attn_mask, float("-inf"))
        else:
            attn_output_weights += attn_mask

    if key_padding_mask is not None:
        attn_output_weights = attn_output_weights.view(bsz, num_heads, tgt_len, src_len)
        attn_output_weights = attn_output_weights.masked_fill(
            key_padding_mask.unsqueeze(1).unsqueeze(2),
            float("-inf"),
        )
        attn_output_weights = attn_output_weights.view(bsz * num_heads, tgt_len, src_len)

    attn_output_weights_raw = attn_output_weights
    attn_output_weights = softmax(attn_output_weights, dim=-1)
    attn_output_weights = dropout(attn_output_weights, p=dropout_p, training=training)

    attn_output = torch.bmm(attn_output_weights, v)
    assert list(attn_output.size()) == [bsz * num_heads, tgt_len, head_dim]
    attn_output = attn_output.transpose(0, 1).contiguous().view(tgt_len, bsz, embed_dim)
    attn_output = linear(attn_output, out_proj_weight, out_proj_bias)

    if need_weights:
        if need_raw:

            attn_output_weights_raw = attn_output_weights_raw.view(bsz, num_heads, tgt_len, src_len)
            return attn_output, attn_output_weights_raw

            # attn_output_weights = attn_output_weights.view(bsz, num_heads, tgt_len, src_len)
            # return attn_output, attn_output_weights.sum(dim=1) / num_heads, attn_output_weights_raw, attn_output_weights_raw.sum(dim=1) / num_heads
        else:
            # average attention weights over heads
            attn_output_weights = attn_output_weights.view(bsz, num_heads, tgt_len, src_len)
            return attn_output, attn_output_weights.sum(dim=1) / num_heads
    else:
        return attn_output, None


import torch
from torch import Tensor

# if float(torch.__version__.split('.')[0]) == 0 or (
#         float(torch.__version__.split('.')[0]) == 1 and float(torch.__version__.split('.')[1])) < 9:
#     from torch.nn.modules.linear import _LinearWithBias
# else:
#     from torch.nn.modules.linear import NonDynamicallyQuantizableLinear as _LinearWithBias
from torch.nn.modules.linear import NonDynamicallyQuantizableLinear as _LinearWithBias

from torch.nn.init import xavier_uniform_
from torch.nn.init import constant_
from torch.nn.init import xavier_normal_
from torch.nn.parameter import Parameter
from torch.nn import Module


class MultiheadAttention(Module):
    r"""Allows the model to jointly attend to information
    from different representation subspaces.
    See reference: Attention Is All You Need

    .. math::
        \text{MultiHead}(Q, K, V) = \text{Concat}(head_1,\dots,head_h)W^O
        \text{where} head_i = \text{Attention}(QW_i^Q, KW_i^K, VW_i^V)

    Args:
        embed_dim: total dimension of the model.
        num_heads: parallel attention heads.
        dropout: a Dropout layer on attn_output_weights. Default: 0.0.
        bias: add bias as module parameter. Default: True.
        add_bias_kv: add bias to the key and value sequences at dim=0.
        add_zero_attn: add a new batch of zeros to the key and
                       value sequences at dim=1.
        kdim: total number of features in key. Default: None.
        vdim: total number of features in value. Default: None.

        Note: if kdim and vdim are None, they will be set to embed_dim such that
        query, key, and value have the same number of features.

    Examples::

        >>> multihead_attn = nn.MultiheadAttention(embed_dim, num_heads)
        >>> attn_output, attn_output_weights = multihead_attn(query, key, value)
    """
    bias_k: Optional[torch.Tensor]
    bias_v: Optional[torch.Tensor]

    def __init__(self, embed_dim, num_heads, dropout=0., bias=True, add_bias_kv=False, add_zero_attn=False, kdim=None,
                 vdim=None):
        super(MultiheadAttention, self).__init__()
        self.embed_dim = embed_dim
        self.kdim = kdim if kdim is not None else embed_dim
        self.vdim = vdim if vdim is not None else embed_dim
        self._qkv_same_embed_dim = self.kdim == embed_dim and self.vdim == embed_dim

        self.num_heads = num_heads
        self.dropout = dropout
        self.head_dim = embed_dim // num_heads
        assert self.head_dim * num_heads == self.embed_dim, "embed_dim must be divisible by num_heads"

        if self._qkv_same_embed_dim is False:
            self.q_proj_weight = Parameter(torch.Tensor(embed_dim, embed_dim))
            self.k_proj_weight = Parameter(torch.Tensor(embed_dim, self.kdim))
            self.v_proj_weight = Parameter(torch.Tensor(embed_dim, self.vdim))
            self.register_parameter('in_proj_weight', None)
        else:
            self.in_proj_weight = Parameter(torch.empty(3 * embed_dim, embed_dim))
            self.register_parameter('q_proj_weight', None)
            self.register_parameter('k_proj_weight', None)
            self.register_parameter('v_proj_weight', None)

        if bias:
            self.in_proj_bias = Parameter(torch.empty(3 * embed_dim))
        else:
            self.register_parameter('in_proj_bias', None)
        self.out_proj = _LinearWithBias(embed_dim, embed_dim)

        if add_bias_kv:
            self.bias_k = Parameter(torch.empty(1, 1, embed_dim))
            self.bias_v = Parameter(torch.empty(1, 1, embed_dim))
        else:
            self.bias_k = self.bias_v = None

        self.add_zero_attn = add_zero_attn

        self._reset_parameters()

    def _reset_parameters(self):
        if self._qkv_same_embed_dim:
            xavier_uniform_(self.in_proj_weight)
        else:
            xavier_uniform_(self.q_proj_weight)
            xavier_uniform_(self.k_proj_weight)
            xavier_uniform_(self.v_proj_weight)

        if self.in_proj_bias is not None:
            constant_(self.in_proj_bias, 0.)
            constant_(self.out_proj.bias, 0.)
        if self.bias_k is not None:
            xavier_normal_(self.bias_k)
        if self.bias_v is not None:
            xavier_normal_(self.bias_v)

    def __setstate__(self, state):
        # Support loading old MultiheadAttention checkpoints generated by v1.1.0
        if '_qkv_same_embed_dim' not in state:
            state['_qkv_same_embed_dim'] = True

        super(MultiheadAttention, self).__setstate__(state)

    def forward(self, query, key, value, key_padding_mask=None,
                need_weights=True, need_raw=True, attn_mask=None):
        # type: (Tensor, Tensor, Tensor, Optional[Tensor], bool, Optional[Tensor]) -> Tuple[Tensor, Optional[Tensor]]
        r"""
    Args:
        query, key, value: map a query and a set of key-value pairs to an output.
            See "Attention Is All You Need" for more details.
        key_padding_mask: if provided, specified padding elements in the key will
            be ignored by the attention. When given a binary mask and a value is True,
            the corresponding value on the attention layer will be ignored. When given
            a byte mask and a value is non-zero, the corresponding value on the attention
            layer will be ignored
        need_weights: output attn_output_weights.
        attn_mask: 2D or 3D mask that prevents attention to certain positions. A 2D mask will be broadcasted for all
            the batches while a 3D mask allows to specify a different mask for the entries of each batch.

    Shape:
        - Inputs:
        - query: :math:`(L, N, E)` where L is the target sequence length, N is the batch size, E is
          the embedding dimension.
        - key: :math:`(S, N, E)`, where S is the source sequence length, N is the batch size, E is
          the embedding dimension.
        - value: :math:`(S, N, E)` where S is the source sequence length, N is the batch size, E is
          the embedding dimension.
        - key_padding_mask: :math:`(N, S)` where N is the batch size, S is the source sequence length.
          If a ByteTensor is provided, the non-zero positions will be ignored while the position
          with the zero positions will be unchanged. If a BoolTensor is provided, the positions with the
          value of ``True`` will be ignored while the position with the value of ``False`` will be unchanged.
        - attn_mask: 2D mask :math:`(L, S)` where L is the target sequence length, S is the source sequence length.
          3D mask :math:`(N*num_heads, L, S)` where N is the batch size, L is the target sequence length,
          S is the source sequence length. attn_mask ensure that position i is allowed to attend the unmasked
          positions. If a ByteTensor is provided, the non-zero positions are not allowed to attend
          while the zero positions will be unchanged. If a BoolTensor is provided, positions with ``True``
          is not allowed to attend while ``False`` values will be unchanged. If a FloatTensor
          is provided, it will be added to the attention weight.

        - Outputs:
        - attn_output: :math:`(L, N, E)` where L is the target sequence length, N is the batch size,
          E is the embedding dimension.
        - attn_output_weights: :math:`(N, L, S)` where N is the batch size,
          L is the target sequence length, S is the source sequence length.
        """
        if not self._qkv_same_embed_dim:
            return multi_head_attention_forward(
                query, key, value, self.embed_dim, self.num_heads,
                self.in_proj_weight, self.in_proj_bias,
                self.bias_k, self.bias_v, self.add_zero_attn,
                self.dropout, self.out_proj.weight, self.out_proj.bias,
                training=self.training,
                key_padding_mask=key_padding_mask, need_weights=need_weights, need_raw=need_raw,
                attn_mask=attn_mask, use_separate_proj_weight=True,
                q_proj_weight=self.q_proj_weight, k_proj_weight=self.k_proj_weight,
                v_proj_weight=self.v_proj_weight)
        else:
            return multi_head_attention_forward(
                query, key, value, self.embed_dim, self.num_heads,
                self.in_proj_weight, self.in_proj_bias,
                self.bias_k, self.bias_v, self.add_zero_attn,
                self.dropout, self.out_proj.weight, self.out_proj.bias,
                training=self.training,
                key_padding_mask=key_padding_mask, need_weights=need_weights, need_raw=need_raw,
                attn_mask=attn_mask)

In [25]:
def generate_patient_task_codebook(tasks: List) -> dict:
    all_patients_dict = dict()
    for task_name in sorted(tasks):
        # dataset split, combining patients in all tasks
        json_path = f"/dssg/home/acct-clsyzs/clsyzs/xiayujia/CoPaRa/coattention_prognosis/" \
                    f"splits/json_{task_name.lower()}/"
        with open(os.path.join(json_path, f"fold_0.json"), "r") as f:
            task_patients_json = json.load(f)
        patient_names = task_patients_json["train"] + task_patients_json["val"]
        task_dict = dict(zip(patient_names, [task_name] * len(patient_names)))
        all_patients_dict.update(task_dict)
    return all_patients_dict

In [26]:
def cal_task_metrics_infer(tasks: List, patient_id: List, censorships: np.ndarray, fold: int,
                           event_times: np.ndarray, risk_scores: np.ndarray, codebook_dict: dict):
    all_c_indexes = []
    all_c_indexes_dict = {}
    for task in tasks:
        all_patients = []
        patient_indicator = []
        for patient in patient_id:
            task_patient = codebook_dict[patient]
            if task_patient == task:
                patient_indicator.append(True)
                all_patients.append(patient)
            else:
                patient_indicator.append(False)

        selected_censorship = censorships[np.array(patient_indicator)]
        selected_event_times = event_times[np.array(patient_indicator)]
        selected_risk_scores = risk_scores[np.array(patient_indicator)]

        patients_info = pd.DataFrame({"fold": [fold] * len(all_patients)})
        event_pd = pd.DataFrame(1 - selected_censorship, columns=["event"])
        patient_pd = pd.DataFrame(all_patients, columns=["patient"])
        risk_pd = pd.DataFrame(selected_risk_scores, columns=["risk"])
        time_pd = pd.DataFrame(selected_event_times, columns=["time"])
        save_pd = pd.concat([patients_info, patient_pd, risk_pd, event_pd, time_pd], axis=1)
        
        c_index_task = concordance_index_censored(save_pd["event"].values.astype(bool),
                                                  save_pd["time"].values, save_pd["risk"].values,
                                                  tied_tol=1e-08)[0]
        all_c_indexes_dict[task] = c_index_task
        all_c_indexes.append(c_index_task)
    average_c_index = np.mean(np.array(all_c_indexes))

    return all_c_indexes_dict, average_c_index

In [27]:
seed = 330
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
random.seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [28]:
fold = 1
task = ["SHRJ", "LUNG", "BRAT"]
patient_codebook = generate_patient_task_codebook(task)
dataset = CoAttnData(fold=fold, task=task, codebook_dict=patient_codebook, mode="val")
loader = DataLoader(dataset=dataset, batch_size=1, shuffle=False, num_workers=0, drop_last=False)

print(f"Use the checkpoint from fold {fold}!")
ckpt_path = f"/dssg/home/acct-clsyzs/clsyzs/xiayujia/CoPaRa/coattention-pathomics-radiomics/union_training/ckpts/fold_{fold}.pth"
model = PaRaMILUnionInfer(tasks=task, codebook_dict=patient_codebook).to(device)
checkpoint = torch.load(ckpt_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

# validation stage
all_patients_val = []
with torch.no_grad():
    model.eval()
    all_risk_scores_val = np.zeros((len(loader)))
    all_censorships_val = np.zeros((len(loader)))
    all_event_times_val = np.zeros((len(loader)))

    for batch_idx, data in enumerate(loader):
        patho, radio_shape, radio_order, radio_texture, radio_log_sigma, radio_wavelet, radio_cnn, \
        surv_discrete, surv_time, censor, patient_name = data

        data_WSI = patho.to(device)
        data_omic1 = radio_shape.type(torch.FloatTensor).to(device)
        data_omic2 = radio_order.type(torch.FloatTensor).to(device)
        data_omic3 = radio_texture.type(torch.FloatTensor).to(device)
        data_omic4 = radio_log_sigma.type(torch.FloatTensor).to(device)
        data_omic5 = radio_wavelet.type(torch.FloatTensor).to(device)
        data_omic6 = radio_cnn.type(torch.FloatTensor).to(device)
        surv_discrete = surv_discrete.type(torch.LongTensor).to(device)
        censor = censor.type(torch.FloatTensor).to(device)

        hazards, S, _, coattn, pathomics, radiomics = model(x_path=data_WSI, x_omic1=data_omic1, x_omic2=data_omic2,
                                                            x_omic3=data_omic3, x_omic4=data_omic4, x_omic5=data_omic5,
                                                            x_omic6=data_omic6, patient_name=patient_name)
        all_patients_val.append(patient_name[0])

        risk = -torch.sum(S, dim=1).detach().cpu().numpy()
        all_risk_scores_val[batch_idx] = risk
        all_censorships_val[batch_idx] = censor.item()
        all_event_times_val[batch_idx] = surv_time.item()

        del patho, radio_shape, radio_order, radio_texture, radio_log_sigma, radio_wavelet, radio_cnn

    c_index_val_dict, c_index_val = cal_task_metrics_infer(tasks=task,
                                                           patient_id=all_patients_val,
                                                           fold=fold,
                                                           censorships=all_censorships_val,
                                                           event_times=all_event_times_val,
                                                           risk_scores=all_risk_scores_val,
                                                           codebook_dict=patient_codebook)

    print(f"[Validation] Fold {fold}, c-index {c_index_val:.4f}, task-level {c_index_val_dict}.")


Use the checkpoint from fold 1!
[Validation] Fold 1, c-index 0.7238, task-level {'SHRJ': 0.8112286411716843, 'LUNG': 0.6476345840130505, 'BRAT': 0.712401055408971}.
